In [1]:
import pandas as pd 
import numpy as np
pd.options.mode.chained_assignment = None
data = pd.read_csv("~/Downloads/Forecast Details - Campaign Program.csv")
pd.DataFrame(data)
budgetdata = pd.read_csv("~/Documents/Budget Tracking/Budget Totals.csv")
pd.DataFrame(budgetdata)
data['Date Pulled'] = pd.datetime.now().date()
data.columns = ["Function", "Budget_Name", "Campaign", "Marketing_Activity", 
                 "Marketing_Activity_Type", "Line_Item", "Activity_Description", 
                 "Quarter", "Spent", "Forecast", "Date_Pulled"]
data['Function'] = np.where(data['Budget_Name'].str.match('532 - Campaign - Cross-Program Digital'),
                            'GDC Digital', data['Function'])
data['Campaign'] = data['Campaign'].str.slice(10,)
data['Campaign'] = data['Campaign'].str.replace('-',' ')
data['Forecast'] = np.where(data['Function'].str.match('GDC Digital') & 
                            data['Campaign'].str.match('Agile Integration'), data['Forecast']*.3746,
                            data['Forecast'])
data['Forecast'] = np.where(data['Function'].str.match('GDC Digital') & 
                            data['Campaign'].str.match('Cloud Native App Dev'), data['Forecast']*4,
                            data['Forecast'])
data['Spent'] = np.where(data['Function'].str.match('GDC Digital') & 
                            data['Campaign'].str.match('Agile Integration'), data['Spent']*.3746,
                            data['Spent'])
data['Spent'] = np.where(data['Function'].str.match('GDC Digital') & 
                            data['Campaign'].str.match('Cloud Native App Dev'), data['Spent']*4,
                            data['Spent'])
data['Negative_Forecast_Tag'] = 'Negative Forecast'
data['Negative_Forecast_Tag'][data['Forecast'] < 0] = 'Negative Forecast'
data['Negative_Forecast_Tag'][data['Forecast'] >= 0] = 'Positive Forecast'
data['Negative_Spent_Tag'] = 'Negative Spent'
data['Negative_Spent_Tag'][data['Spent'] < 0] = 'Negative Spent'
data['Negative_Spent_Tag'][data['Spent'] >= 0] = 'Positive Spent'
data['OpenShift_Tag'] = np.where(data['Activity_Description'].str.contains('openshift|ocp|container|containers',
                                                                           na=False, case=False),'OpenShift', 
                                np.where(data['Line_Item'].str.contains('openshift|ocp|container|containers',
                                na=False, case=False), 'OpenShift','Middleware'))
duplicate = data[data.duplicated(['Function', 'Budget_Name', 'Campaign', 'Marketing_Activity', 
                 'Marketing_Activity_Type', 'Line_Item', 'Activity_Description'])]
unique = data[~data.duplicated(['Function', 'Budget_Name', 'Campaign', 'Marketing_Activity', 
                 'Marketing_Activity_Type', 'Line_Item', 'Activity_Description'])]
duplicatenotzero = duplicate[~((duplicate['Spent']==0) & (duplicate['Forecast']==0))]
uniquenotzero = unique[~((unique['Spent']==0) & (unique['Forecast']==0))]
allnotzero = pd.concat([duplicatenotzero, uniquenotzero])
duplicatezero = duplicate[(duplicate['Spent']==0) & (duplicate['Forecast']==0)]
uniquezero = unique[(unique['Spent']==0) & (unique['Forecast']==0)]
allzero = pd.concat([duplicatezero, uniquezero])
allzeroduplicate = allzero[allzero.duplicated(['Function', 'Budget_Name', 'Campaign', 'Marketing_Activity', 
                 'Marketing_Activity_Type', 'Line_Item', 'Activity_Description'])]
allzerounique = allzero[~allzero.duplicated(['Function', 'Budget_Name', 'Campaign', 'Marketing_Activity', 
                 'Marketing_Activity_Type', 'Line_Item', 'Activity_Description'])]
finaldata = pd.concat([allzerounique, allnotzero])
with pd.ExcelWriter('Budget Tracking Data Python.xlsx', engine='openpyxl', mode='a') as writer:
    finaldata.to_excel(writer, sheet_name=str(pd.datetime.now().date()), index=False)